# TensorFlow Lite Conversion

In [3]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from models import ArcFaceModel
import tensorflow as tf
import os
import logging
import numpy as np
from pathlib import Path
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model, load_model
from layers import ArcMarginPenaltyLogists
from losses import softmax_loss
from glob import glob
import dataset

tf.get_logger().setLevel(logging.ERROR)
# os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [4]:
!echo $CUDA_VISIBLE_DEVICES

1


In [5]:
### IJB-C Dataset

# batch_size = 128
# input_size = 112
# embd_shape = 512
# head_type = 'ArcHead'
# backbone_type = 'MobileNetV2'
# w_decay=5e-4
# num_classes = 3584 
# base_lr = 0.01
# dataset_len = 13033 
# epochs = 100
# steps_per_epoch = dataset_len // batch_size

### MS1M dataset

batch_size = 128 # Initially 128
input_size = 112
embd_shape = 512
head_type = 'ArcHead'
backbone_type = 'MobileNetV2' # MobileNetV2, ResNet50, EfficientNetLite0~6
w_decay=5e-4
num_classes = 85742 
dataset_len = 5822653 
base_lr = 0.01 # initially 0.01
epochs = 20
save_steps = 1000
train_size = int(0.8 * dataset_len)
steps_per_epoch = train_size // batch_size
val_size = dataset_len - train_size
validation_steps = val_size // batch_size
steps = 1
is_ccrop=False
binary_img=True
is_Adam = False

dgx = True

In [6]:
from glob import glob
if dgx:
    base_dir = "/raid/workspace/jbpark/FaceRecognition/checkpoints/wo_tfidentity/"
#     base_dir = "/raid/workspace/jbpark/FaceRecognition/checkpoints/wo_tfidentity/"
#     save_name = "ms1m_mobilenet_check/SGD/*"
    save_name = "ms1m_mobilenet_check/SGD/e*"
else:
    base_dir = "/hd/jbpark/models/checkpoints/w_tfidentity/"
    save_name = "ms1m_check/SGD/*"
file_list = []
for files in glob(base_dir+save_name):
    file_list.append(files.split('/')[-1].split('l_')[-1])
file_list.sort()

load_file_name = []
for files in glob(base_dir+save_name):
    if file_list[0] == files.split('/')[-1].split('l_')[-1]:
        load_file_name = files
best_checkpoint = load_file_name.split('.data')[0]
initial_epoch = int(load_file_name.split('e_')[-1].split('_')[0])-1
print(initial_epoch)
print(best_checkpoint)

4
/raid/workspace/jbpark/FaceRecognition/checkpoints/wo_tfidentity/ms1m_mobilenet_check/SGD/e_5_l_16.65623664855957.ckpt


In [7]:
accuracy = file_list[0].split('.')[0]
print(accuracy)

16


### Without Quantization

In [6]:
weight_file = best_checkpoint

model = ArcFaceModel(size=input_size,
                         backbone_type=backbone_type,
                         training=False)
model.load_weights(weight_file)

# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TF Lite model.
if dgx:
    Path('/raid/workspace/jbpark/FaceRecognition/tflite/').mkdir(parents=True, exist_ok=True)
    with tf.io.gfile.GFile('/raid/workspace/jbpark/FaceRecognition/tflite/ms1m_acc_'+str(accuracy)+'.tflite', 'wb') as f:
        f.write(tflite_model)
else:
    Path('/hd/jbpark/models/tflite/').mkdir(parents=True, exist_ok=True)
    with tf.io.gfile.GFile('/hd/jbpark/models/tflite/ms1m_acc_'+str(accuracy)+'.tflite', 'wb') as f:
        f.write(tflite_model)

### With Qunatization

In [10]:
# train_data_dir = "/raid/workspace/jbpark/IJB-C_Asian/"
# tfrecord_name = train_data_dir+'ijbc_bin.tfrecord'
if dgx:
    train_data_dir = "/raid/workspace/jbpark/ms1m/"
    tfrecord_name = train_data_dir+'ms1m_bin.tfrecord'
else:
    train_data_dir = "/hd/jbpark/dataset/ms1m/"
    tfrecord_name = train_data_dir+'ms1m_bin.tfrecord'


train_dataset = dataset.load_representative_dataset(
    tfrecord_name, batch_size=1, train_size=train_size, binary_img=binary_img,
    is_ccrop=is_ccrop)

In [11]:
def representative_data_gen():
    count=0
    for input_value in train_dataset:
        count+=1
        if count > 100: return
        yield [input_value]

In [12]:
for data in representative_data_gen():
    print(data)

[<tf.Tensor: shape=(1, 112, 112, 3), dtype=float32, numpy=
array([[[[0.7063765 , 0.75569636, 0.5425999 ],
         [0.7029451 , 0.752265  , 0.5378199 ],
         [0.7170383 , 0.76635814, 0.5518167 ],
         ...,
         [0.1640297 , 0.15422353, 0.14864615],
         [0.12568718, 0.11996531, 0.11619889],
         [0.09616819, 0.09507968, 0.09092795]],

        [[0.7265052 , 0.775825  , 0.5640579 ],
         [0.72693413, 0.776254  , 0.5644868 ],
         [0.72258365, 0.77190346, 0.55788225],
         ...,
         [0.17410937, 0.16051751, 0.15605754],
         [0.13290226, 0.12535016, 0.12211354],
         [0.10217309, 0.10016948, 0.09616224]],

        [[0.7535736 , 0.801882  , 0.59530693],
         [0.7575622 , 0.8068146 , 0.5974075 ],
         [0.74789   , 0.79720986, 0.5835932 ],
         ...,
         [0.19737837, 0.1810219 , 0.17764078],
         [0.15627848, 0.14623149, 0.14407374],
         [0.12565655, 0.12102318, 0.11789253]],

        ...,

        [[0.7181559 , 0.7625438 ,

In [13]:
weight_file = best_checkpoint

model = ArcFaceModel(size=input_size,
                         backbone_type=backbone_type,
                         training=False)
model.load_weights(weight_file)

# Convert the model.
model.input.set_shape((1,) + model.input.shape[1:])
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
converter.target_spec.supported_types = [tf.int8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_model_quant = converter.convert()

# Save the TF Lite model.
if dgx:
    Path('/raid/workspace/jbpark/FaceRecognition/tflite/').mkdir(parents=True, exist_ok=True)
    with tf.io.gfile.GFile('/raid/workspace/jbpark/FaceRecognition/tflite/ms1m_acc_'+str(accuracy)+'.tflite', 'wb') as f:
        f.write(tflite_model_quant)
else:
    Path('/hd/jbpark/models/tflite/').mkdir(parents=True, exist_ok=True)
    with tf.io.gfile.GFile('/hd/jbpark/models/tflite/ms1m_acc_'+str(accuracy)+'.tflite', 'wb') as f:
        f.write(tflite_model_quant)

9412608/9406464 [==============================] - 0s 0us/step


### Input / Output Check

In [8]:
import numpy as np
import tensorflow as tf

# Load the TFLite model and allocate tensors.
if dgx:
    interpreter = tf.lite.Interpreter(model_path='/raid/workspace/jbpark/FaceRecognition/tflite/ms1m_mobilenetv2_'+str(accuracy)+'_quant.tflite')
else:
    interpreter = tf.lite.Interpreter(model_path='/hd/jbpark/models/tflite/ms1m_acc_'+str(accuracy)+'.tflite')
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
input_details

[{'name': 'input_image',
  'index': 182,
  'shape': array([  1, 112, 112,   3], dtype=int32),
  'shape_signature': array([  1, 112, 112,   3], dtype=int32),
  'dtype': numpy.uint8,
  'quantization': (0.003933776170015335, 0),
  'quantization_parameters': {'scales': array([0.00393378], dtype=float32),
   'zero_points': array([0], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [9]:
output_details = interpreter.get_output_details()
output_details

[{'name': 'Identity',
  'index': 183,
  'shape': array([  1, 512], dtype=int32),
  'shape_signature': array([  1, 512], dtype=int32),
  'dtype': numpy.uint8,
  'quantization': (0.04355792701244354, 127),
  'quantization_parameters': {'scales': array([0.04355793], dtype=float32),
   'zero_points': array([127], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

### Extract Embedding Vectors & Create Classifier Training Dataset

In [10]:
import cv2
import numpy as np
from utils import l2_norm
from glob import glob
from tqdm import tqdm

dataset_path = "/raid/workspace/jbpark/IJB-C_Asian/Aligned/"

In [11]:
import os 
from tqdm import tqdm
id_list = os.listdir(dataset_path)
id_list.sort()
source_id = []
for id_name in tqdm(id_list):
    source_id.append(int(id_list.index(id_name)))
len(np.unique(np.array(source_id)))

100%|██████████| 3585/3585 [00:00<00:00, 45813.06it/s]


3585

In [12]:
subjects = id_list
label_int = source_id
embed_list = []
label_list = []
for subject in tqdm(subjects):
#     print("[*] Encode {} to Embedding Vector ({})".format(subject,embd_shape))
    img_paths = glob(dataset_path+subject+"/*")
    for img_path in img_paths:
#         print("[*] Encode {} to ./output_embeds.npy".format(img))
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (112,112))
        img = img.astype(np.uint8) 
        # print("len(img.shape): "+str(len(img.shape))+ ", img.shape = "+str(img.shape))
        if len(img.shape) == 3:
            img = np.expand_dims(img, 0)

        interpreter.set_tensor(input_details[0]['index'], img)
        interpreter.invoke()
        output_data = interpreter.get_tensor(output_details[0]['index'])
    #     print("len(img.shape): "+str(len(img.shape))+ ", img.shape = "+str(img.shape))
        embeds = l2_norm(output_data)
    #     embeds.shape
    #     embeds
        embed_list.append(embeds[0])
        label_list.append(label_int[subjects.index(subject)])
embed_list = np.asarray(embed_list)
label_list = np.asarray(label_list)
# embed_list

100%|██████████| 3585/3585 [47:24<00:00,  1.26it/s]  


In [13]:
save_path = "/raid/workspace/jbpark/IJB-C_Asian/Quant/"
np.save(save_path+'ijbc_embed_vectors.npy', embed_list)
np.save(save_path+'ijbc_labels.npy', label_list)

In [14]:
dataset_path = "/raid/workspace/jbpark/IJB-C_Asian/Aligned/"
id_list = os.listdir(dataset_path)
id_list.sort()

unique_labels = np.unique(label_list)

centroid_list = []
for i in tqdm(unique_labels):
    temp_list = np.where(label_list==i)[0]
    centroid = np.average(embed_list[temp_list],axis=0)
    centroid_list.append(centroid)
np.array(centroid_list).shape

100%|██████████| 3585/3585 [00:00<00:00, 32525.94it/s]


(3585, 512)

In [15]:
id_list

['2 Chainz',
 '50 Cent',
 'A. P. J. Abdul Kalam',
 'A.K. Antony',
 'AF0301',
 'AF0302',
 'AF0303',
 'AF0304',
 'AF0305',
 'AF0306',
 'AF0307',
 'AF0308',
 'AF0309',
 'AF0310',
 'AF0311',
 'AF0312',
 'AF0313',
 'AF0314',
 'AF0315',
 'AF0316',
 'AF0317',
 'AF0318',
 'AF0319',
 'AF0320',
 'AF0321',
 'AF0322',
 'AF0323',
 'AF0325',
 'AF0326',
 'AF0327',
 'AF0328',
 'AF0329',
 'AF0330',
 'AF0331',
 'AF0332',
 'AF0333',
 'AF0334',
 'AF0336',
 'AJ Lee',
 'AM0301',
 'AM0302',
 'AM0303',
 'AM0304',
 'AM0305',
 'AM0306',
 'AM0307',
 'AM0308',
 'AM0309',
 'AM0310',
 'AM0311',
 'AM0312',
 'AM0313',
 'AM0314',
 'AM0315',
 'AM0316',
 'AM0317',
 'AM0318',
 'AM0319',
 'Aaron Paul',
 'Aaron Sorkin',
 'Abbas Kiarostami',
 'Abby Wambach',
 'Abdallah Djaballah',
 'Abdel Fattah el-Sisi',
 'Abdelilah Benkirane',
 'Abdellah Taïa',
 'Abdelmalek Sellal',
 'Abderrahmane Sissako',
 'Abdou Diouf',
 'Abdoulaye Wade',
 'Abdul Jabbar',
 'Abdul Malik Baloch',
 'Abdullah Abdullah',
 'Abdullah Bin Zayed al-Nahyan',
 'A

In [16]:
save_path = "/raid/workspace/jbpark/IJB-C_Asian/Quant/"
np.savetxt(save_path+'quant_id_list.txt', id_list, fmt='%s', delimiter=',')
np.savetxt(save_path+'quant_centroid_list.csv', centroid_list, fmt='%f', delimiter=',')